In [20]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
import scipy.io as sio
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import os

from torch.utils.data import DataLoader, Dataset, TensorDataset
from utils.EmoPainDataset import KinematicsDataset, SlidingWindowDataset, create_weighted_sampler
from utils.models import KinematicsTransformer, KinematicsLSTM, FocalLoss
from sklearn.metrics import balanced_accuracy_score
from utils.training import train_transformer, pretrain_transformer

filepath='EmoPainData/P.feather'

sampling_frequency = 60

In [27]:
# read dataframe
df=pd.read_feather(filepath)
#df=pd.concat([pd.read_feather('EmoPainData/P.feather'), pd.read_feather('EmoPainData/C.feather')], axis=0, ignore_index=True)
df.columns = [feature[:-4] if (feature.find('nan')!=-1) else feature for feature in df.columns] # drop 'nan' in column names
# calculate a soft probability of protective behaviors
behavior_ratings=[feature for feature in df.columns if feature.find('Rater')!=-1]
df.insert(df.shape[-1], 'behavior prob', (df[behavior_ratings]/2).mean(axis=1)) # divided by 2 because the max label is 2
# calculate a binary label of protective behaviors based on the majority voting used in the original paper and Wang et. al 
binary_labels_per_rater, PB_label=pd.DataFrame(), pd.DataFrame()
for rater_num in range(1,5):    # loop through all 4 raters
    behavior_ratings=[feature for feature in df.columns if feature.find('Rater '+str(rater_num))!=-1]
    # the maximum label of each frame given by each rater
    pb = df[behavior_ratings].max(axis=1)
    PB_label.insert(rater_num-1,str(rater_num), pb)
    # create a binary label based on majority voting of all 4 raters
    binary_label = ((df[behavior_ratings]==1) | (df[behavior_ratings]==2)).max(axis=1)
    binary_labels_per_rater.insert(rater_num-1,str(rater_num), binary_label)
majority_voting_binary_label = (binary_labels_per_rater.sum(axis=1)>=2).astype(int)
df.insert(df.shape[-1], 'behavior binary', majority_voting_binary_label)
df.insert(df.shape[-1], 'PB prob', PB_label.mean(axis=1)/2)

In [4]:
print(df.columns.to_numpy())

['Hip' 'LeftUpperLeg' 'LeftLowerLeg' 'LeftAnkle' 'LeftHeel' 'LeftToes'
 'RightUpperLeg' 'RightLowerLeg' 'RightAnkle' 'RightHeel' 'RightToes'
 'Spine' 'Spine 1' 'LeftShoulder' 'LeftUpperArm' 'LeftLowerArm'
 'LeftWrist' 'LeftFingertip' 'RightShoulder' 'RightUpperArm' 'RightArm'
 'RightWrist' 'RightFingertip' 'Neck' 'Head' 'Crown'
 'Rectified EMG Probe 1:Right Lower' 'Rectified EMG Probe 2:Left Lower'
 'Rectified EMG Probe 3:Right Upper' 'Recfified EMG Probe 4:Left Upper'
 'Rater 1- Guarding/Stiffness' 'Rater 1- Hesitation'
 'Rater 1- Support/Bracing' 'Rater 1- Jerky Motion' 'Rater 1 - Limping'
 'Rater 1 - Rubbing/Stimulation' 'Rater 1 - Other'
 'Rater 2- Guarding/Stiffness' 'Rater 2- Hesitation'
 'Rater 2- Support/Bracing' 'Rater 2- Jerky Motion' 'Rater 2 - Limping'
 'Rater 2 - Rubbing/Stimulation' 'Rater 2 - Other'
 'Rater 3- Guarding/Stiffness' 'Rater 3- Hesitation'
 'Rater 3- Support/Bracing' 'Rater 3- Jerky Motion' 'Rater 3 - Limping'
 'Rater 3 - Rubbing/Stimulation' 'Rater 3 - Other

In [5]:
behavior_ratings=[feature for feature in df.columns if feature.find('Rater')!=-1]
print(df.shape)
print((df[behavior_ratings]==0).sum(axis=0)/df.shape[0])
print((df[behavior_ratings]==1).sum(axis=0)/(df[behavior_ratings]==2).sum(axis=0))

(473210, 113)
Rater 1- Guarding/Stiffness      0.962782
Rater 1- Hesitation              0.997483
Rater 1- Support/Bracing         0.991701
Rater 1- Jerky Motion            0.992327
Rater 1 - Limping                0.992158
Rater 1 - Rubbing/Stimulation    0.997969
Rater 1 - Other                  0.998599
Rater 2- Guarding/Stiffness      0.782498
Rater 2- Hesitation              0.997099
Rater 2- Support/Bracing         0.963226
Rater 2- Jerky Motion            0.952554
Rater 2 - Limping                0.948040
Rater 2 - Rubbing/Stimulation    0.987834
Rater 2 - Other                  1.000000
Rater 3- Guarding/Stiffness      0.903559
Rater 3- Hesitation              0.998614
Rater 3- Support/Bracing         0.970436
Rater 3- Jerky Motion            0.984527
Rater 3 - Limping                0.981625
Rater 3 - Rubbing/Stimulation    0.997394
Rater 3 - Other                  0.997416
Rater 4- Guarding/Stiffness      0.708066
Rater 4- Hesitation              0.989816
Rater 4- Support/Bra

In [26]:
from utils.EmoPainDataset import KinematicsDataset, SlidingWindowDataset

# Load the dataset
    # indicate which features to use
subjects = df.Subject.unique()
leave_out_subject = subjects[0]
kinematic_features=[feature for feature in df.columns if feature.find('Energy')!=-1 or feature.find('Angle')!=-1 ]
behavior_ratings=[feature for feature in df.columns if feature.find('Rater')!=-1]
PB_prob='PB prob'
behavior_prob='behavior prob'
behavior_binary='behavior binary'
target_features = [behavior_binary] # target
df = df.dropna(subset=kinematic_features,)  # the first sample for each subject is nan for energy features; so drop them
#
window_size, step_size = int(sampling_frequency*3), int(sampling_frequency*0.75)  # experiment parameters

dataset = SlidingWindowDataset(df, leave_out_subject, kinematic_features, target_features, 'Subject', window_size, step_size)
test_features, test_targets = dataset.get_test_data()
print(test_features.shape, test_targets.shape)

torch.Size([660, 180, 26]) torch.Size([660, 1])


In [ ]:
#   1. majority voting of maximum labels (the label of a frame from a single rater is the max label they give across all 7 PBs)
    #   a. per-frame prediction
    #   b. per-sequence prediction (majority voting of frames)

#   2. mean scores as labels across all raters and all categories, divided by 2
    #   a. per-frame prediction
    #   b. per-sequence prediction (mean of frames)



#   Pre-training schemes
    #   1. Transformer
        #   a. Train the encoder and decoder to predict the activity at each frame
        #   b. Train the encoder and decoder to 
        #   c. Evaluate the pre-trained model on the test set
    #   2.  LSTM
    #   3.  Point-Net


#   Fine-tuning schemes

In [ ]:
import seaborn as sns
sns.boxplot(df[kinematic_features])

In [160]:
from utils.models import SinusoidalPositionalEncoding

PE = SinusoidalPositionalEncoding(2)
PE.forward(torch.zeros(1, 10, 2))

tensor([[[ 0.0000,  0.7071],
         [ 0.5950,  0.3821],
         [ 0.6430, -0.2943],
         [ 0.0998, -0.7000],
         [-0.5351, -0.4622],
         [-0.6781,  0.2006],
         [-0.1976,  0.6789],
         [ 0.4646,  0.5331],
         [ 0.6996, -0.1029],
         [ 0.2914, -0.6443]]])

In [15]:
# Hyperparameters
input_dim = test_features.shape[-1]           # Number of features in each kinematics sample
num_classes = 2
seq_len = window_size       # Sequence length (window length)
embed_dim = 256             # Embedding dimension
num_heads = 4               # Number of attention heads
num_layers = 4              # Number of transformer layers
dropout = 0.1               # Dropout rate
lr = 1e-2                   # Learning rate
masking_prob = 0.2          # Probability of masking each feature
pretrain_epochs = 10        # Pretraining epochs
fine_tune_epochs = 10       # Fine-tuning epochs
batch_size = 64             # Batch size
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Initialize model
model = KinematicsTransformer(input_dim, num_classes, seq_len, embed_dim, num_heads, num_layers, dropout)
#model = KinematicsLSTM(input_dim, embed_dim, num_classes, num_layers, dropout, bidirectional=True)
torch.cuda.empty_cache()
# Create a weighted sampler
weighted_sampler = create_weighted_sampler(dataset, num_bins=2)

# DataLoader with weighted sampling
train_loader = DataLoader(dataset, batch_size=batch_size, sampler=weighted_sampler)
test_features, test_targets = dataset.get_test_data()
val_loader = DataLoader(TensorDataset(test_features, test_targets), batch_size=batch_size)

In [16]:
# Pretrain the transformer
torch.cuda.empty_cache()
pretrain_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
pretrain_loader = train_loader
model.pretrain=True
pretrain_transformer(model, pretrain_loader, pretrain_epochs, lr, masking_prob, device, parameters=model.backbone.parameters())

Pretraining Epoch 1/10, Loss: 1.2649
Pretraining Epoch 2/10, Loss: 1.3565
Pretraining Epoch 3/10, Loss: 1.4396
Pretraining Epoch 4/10, Loss: 1.2030
Pretraining Epoch 5/10, Loss: 1.3619
Pretraining Epoch 6/10, Loss: 1.3794
Pretraining Epoch 7/10, Loss: 1.3489
Pretraining Epoch 8/10, Loss: 1.4754
Pretraining Epoch 9/10, Loss: 1.2972
Pretraining Epoch 10/10, Loss: 1.3598


In [17]:
# Fine-tune model
model.pretrain=False
train_transformer(model, train_loader, val_loader, fine_tune_epochs, lr, device,parameters=model.parameters())

Epoch 1/10, Train Loss: 0.6759, Val Loss: 0.6758, Val Accuracy: 0.6628
Epoch 2/10, Train Loss: 0.6751, Val Loss: 0.6758, Val Accuracy: 0.6628
Epoch 3/10, Train Loss: 0.6758, Val Loss: 0.6758, Val Accuracy: 0.6628
Epoch 4/10, Train Loss: 0.6758, Val Loss: 0.6758, Val Accuracy: 0.6628
Epoch 5/10, Train Loss: 0.6758, Val Loss: 0.6758, Val Accuracy: 0.6628
Epoch 6/10, Train Loss: 0.6747, Val Loss: 0.6758, Val Accuracy: 0.6628
Epoch 7/10, Train Loss: 0.6749, Val Loss: 0.6758, Val Accuracy: 0.6628
Epoch 8/10, Train Loss: 0.6759, Val Loss: 0.6758, Val Accuracy: 0.6628
Epoch 9/10, Train Loss: 0.6754, Val Loss: 0.6758, Val Accuracy: 0.6628
Epoch 10/10, Train Loss: 0.6753, Val Loss: 0.6758, Val Accuracy: 0.6628


In [13]:
model.to(device)
model.eval()
#test_targets=test_targets.to(device)
predictions=model(test_features.to(device))
#balanced_accuracy_score(test_targets.mean(dim=-2) > 0.5, predictions.mean(dim=-2).cpu().detach() > 0.5)

In [76]:
X, Y =[], []
for x,y in train_loader:
    X.append(x)
    Y.append(y)

X = torch.cat(X)
Y = torch.cat(Y)

In [77]:
(Y == 1).sum()/Y.flatten().shape[0]

tensor(0.2430)

In [78]:
Y.mean(dim=-2)


tensor([[0.4500],
        [0.9000],
        [0.6556],
        ...,
        [0.0000],
        [1.0000],
        [0.0000]])

In [ ]:
x,y=dataset.get_train_data()
import matplotlib.pyplot as plt
plt.hist(torch.max(y, dim=1)[0])